In [72]:
#Snorkel
import re
import pandas as pd
from snorkel.labeling import LabelingFunction
from snorkel.preprocess import preprocessor
from snorkel.labeling import PandasLFApplier
from snorkel.labeling.model import LabelModel
from snorkel.labeling import LFAnalysis
from snorkel.labeling import filter_unlabeled_dataframe
from snorkel.labeling import labeling_function

pd.set_option('display.max_colwidth', None)

## Labelling Functions (LFs)

In [154]:
POSITIVE = 1
NEGATIVE = 0
ABSTAIN = -1

categories = [POSITIVE, NEGATIVE, ABSTAIN]

#### Positive Sentiment LFs

- Adjectives
- Verbs

In [160]:
@labeling_function()
def lf_news_good_adjs(x):
    with open('./dicts/final/pos_adj.txt') as file:
        adjectives = [line.rstrip() for line in file]
    
    # Identificação dos adjetivos na frase
    for word in x.title.lower().split():
        if word in adjectives:
            return POSITIVE
    return ABSTAIN

@labeling_function()
def lf_news_good_verbs(x):
    with open('./dicts/pos_verbs.txt') as file:
        verbs = [line.rstrip() for line in file]
    
    for word in x.title.lower().split():
        if word in verbs:
            return POSITIVE
        
    return ABSTAIN

@labeling_function()
def lf_regex_dividendos(x):
    dividend_pattern = r".*pag.*dividendo.*|.*anunc.*dividendo.*|.*distrib.*dividendo.*"
    return POSITIVE if re.search(dividend_pattern, x.title.lower(), flags=re.I) else ABSTAIN

@labeling_function()
def lf_regex_resultado_positivo(x):
    raise_pattern = r"fech.*alta.*|.*abr.*alta.*|.*fech.*pos.*|.*abr.*pos.*|.*estre.*alta.*|.*prev.*alta.*" 
    return POSITIVE if re.search(raise_pattern, x.title.lower(), flags=re.I) else ABSTAIN

#### Negative Sentiment LFs

In [176]:
@labeling_function()
def lf_news_bad_adjs(x):
    # Lista Inicial de Adjetivos 
    with open('./dicts/final/neg_adj.txt') as file:
        adjectives = [line.rstrip() for line in file]

    # Identificação dos adjetivos na frase
    for word in x.title.lower().split():
        if word in adjectives:
            return NEGATIVE

    return ABSTAIN

@labeling_function()
def lf_news_bad_verbs(x):
    with open('./dicts/neg_verbs.txt') as file:
        verbs = [line.rstrip() for line in file]
    
    for word in x.title.lower().split():
        if word in verbs:
            return NEGATIVE
        
    return ABSTAIN

@labeling_function()
def lf_regex_resultado_negativo(x):
    fall_pattern = r"fech.*queda.*|.*abr.*queda.*|.*fech.*neg.*|.*abr.*neg.*|.*prev.*baixa.*|.*prev.*queda.*|.*em.*queda.*" 
    
    if re.search(fall_pattern, x.title.lower(), flags=re.I):
        print ('achou', x.title.lower())
        return NEGATIVE
    else:
        return ABSTAIN
    
    # return NEGATIVE if re.search(fall_pattern, x.title.lower(), flags=re.I) else ABSTAIN

In [177]:
d = {'title': ['ibovespa negativo pior queda', 
               'petrobras anuncia data de divulgação de dividendos',
               'petrobras apresenta melhor resultado da história',
               'petrobras apresenta pior resultado da história',
               'petrobras apresenta resultado insatisfatório',
               'petrobras apresenta resultado neutro',
               'economia em queda no cenário internacional',
               'cash3 divulga lucro de um bilhão mas decepciona acionistas']
}

df = pd.DataFrame(data=d)

In [178]:
# combine all the labeling functions 
lfs = [
       lf_news_good_adjs, 
       lf_news_good_verbs,
       lf_regex_dividendos,
       lf_regex_resultado_positivo,
       lf_news_bad_adjs,
       lf_news_bad_verbs,
       lf_regex_resultado_negativo]

# apply the lfs on the dataframe
applier = PandasLFApplier(lfs=lfs)
L_snorkel = applier.apply(df=df, progress_bar=False)

# apply the label model
label_model = LabelModel(cardinality=len(categories), device='cpu', verbose=False)

# fit on the data
label_model.fit(L_snorkel)

# predict and create the labels
df['label'] = label_model.predict(L=L_snorkel, tie_break_policy='abstain').astype(str)

# convert to classes
dict_map = {'-1': 'NEUTRAL', '1': 'POSITIVE', '0': 'NEGATIVE'}
df['label_class'] = df['label'].map(dict_map)

achou economia em queda no cenário internacional


100%|██████████| 100/100 [00:00<00:00, 1616.64epoch/s]


In [179]:
df.head(10)

,title,label,label_class
0,ibovespa negativo pior queda,0,NEGATIVE
1,petrobras anuncia data de divulgação de dividendos,1,POSITIVE
2,petrobras apresenta melhor resultado da história,1,POSITIVE
3,petrobras apresenta pior resultado da história,0,NEGATIVE
4,petrobras apresenta resultado insatisfatório,0,NEGATIVE
5,petrobras apresenta resultado neutro,-1,NEUTRAL
6,economia em queda no cenário internacional,2,NaN
7,cash3 divulga lucro de um bilhão mas decepciona acionistas,-1,NEUTRAL


In [86]:
# #Filtering out unlabeled data points
# df = df.loc[df.label_class.isin(['POSITIVE', 'NEGATIVE']), :]

# print ('Quantidade Total de Mensagens Rotuladas: ', df.shape[0])

# # find the label counts 
# df['label_class'].value_counts() / df.shape[0] * 100

Quantidade Total de Mensagens Rotuladas:  2


NEGATIVE    50.0
POSITIVE    50.0
Name: label_class, dtype: float64